In [228]:
import requests
import sys
import pandas as pd
import numpy as np
import time
import re
# time.sleep(5)
# page = requests.get('https://www.boxofficemojo.com/release/rl1182631425/?ref_=bo_yld_table_1')
# page_list = requests.get('https://www.boxofficemojo.com/year/2020/?grossesOption=calendarGrosses')
from bs4 import BeautifulSoup
# soup = BeautifulSoup(page.content, 'html.parser')
# print(soup.prettify())
# page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
page = requests.get('https://www.boxofficemojo.com/release/rl2621539073/weekend/')
time.sleep(1)
# soup = BeautifulSoup(page.content, 'html.parser')    

In [229]:
# returns the title of the single movie from single movie url in string form
def find_movie_title(page):
    # page = requests.get('https://www.boxofficemojo.com/release/rl2960360961/?ref_=bo_yld_table_16')
    # time.sleep(1)
    soup = BeautifulSoup(page.content, 'html.parser')    
    # print(soup.prettify())
    title = soup.find('h1', class_="a-size-extra-large")
    return title.string

In [230]:
find_movie_title(page)

'The Fox Hunter'

In [231]:
# returns release date of the single movie in string
def find_release_date(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    release_date = soup.find_all('a', class_='a-link-normal')
    date = ''
    for item in release_date:
        if item['href'][-8:] == 'bo_rl_rl':
            # print('here')
            date = item.text
    
    return date

In [232]:
find_release_date(page)

'Jun 5, 2020'

In [233]:
# returns budget cost or NaN of a movie if no budget cost is listed
def find_budget(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    budget = str(soup.find_all('div', class_='a-section a-spacing-none'))
    # print(budget)
    re_budget = re.findall(r"(?<=Budget</span><span><span class=\"money\">).*?(?=</span>)", budget)
    if re_budget:
        return re_budget[0]
    # print(budget)
    return float('NaN')


In [234]:
find_budget(page)

nan

In [235]:
# returns an array of genres of single movie
def find_genre(page):
    soup = BeautifulSoup(page.content, 'html.parser') 
    span_genre = str(soup.find_all('span')).replace('\n', '')
    span_genre = re.findall(r"(?<=<span>Genres<\/span>, <span>).*?(?=</span>)", span_genre)
    return span_genre[0].split()

In [236]:
find_genre(page)

['Comedy']

In [241]:
# returns gross value (domestic and world wide) in dict with two key-values
def find_all_gross(page):
    output = []
    soup = BeautifulSoup(page.content, 'html.parser') 
    gross = str(soup.find_all('div', class_="a-section a-spacing-none"))
    all_money = soup.select('.a-link-normal > .money')
    if (len(all_money) == 2):
        international = int(all_money[0].text.replace('$', '').replace(',', ''))
        worldwide = int(all_money[1].text.replace('$', '').replace(',', ''))
        domestic = worldwide - international
        return {'domestic_gross': domestic, 'worldwide_gross': worldwide}
    if (len(all_money) < 2):
        worldwide = int(all_money[0].text.replace('$', '').replace(',', ''))
        return {'domestic_gross': worldwide, 'worldwide_gross': worldwide}
    else:
        return {'domestic_gross': float('NaN'), 'world_gross': float('NaN')}

In [242]:
find_all_gross(page)

{'domestic_gross': 1051, 'worldwide_gross': 1051}